In [4]:
from gensim.models import KeyedVectors
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [5]:
model = KeyedVectors.load("300features_40minwords_10context")
num_features = 300

In [30]:
model.wv.syn0

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([[ 0.03144343, -0.02232238, -0.02028987, ..., -0.02405546,
         0.09926601,  0.049621  ],
       [-0.05746495, -0.05549316, -0.11874807, ..., -0.03157035,
         0.05097324,  0.01731471],
       [ 0.00215511, -0.00649544,  0.01320117, ..., -0.0620345 ,
         0.02614006,  0.04123367],
       ...,
       [-0.06553942, -0.01146925,  0.15460777, ...,  0.09524195,
        -0.07421808, -0.00829083],
       [-0.01500735, -0.00073907,  0.00629002, ...,  0.02063346,
        -0.01849476, -0.04222548],
       [-0.06901131, -0.05137084,  0.11262897, ..., -0.06355549,
        -0.07343313, -0.02937378]], dtype=float32)

In [7]:
import pandas as pd
train = pd.read_csv("./datasets/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("./datasets/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("./datasets/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [8]:
def review_to_wordlist(review, remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    
    words = letters_only.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
    return words 

In [9]:
import numpy as np
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features), dtype='float32')
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model[word])
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [10]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype='float32')
    for review in reviews:
        if counter % 1000 == 0:
            print("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter += 1
    return reviewFeatureVecs

In [73]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
print("Creating average feature vecs for text reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 25000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for text reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [78]:
# # Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
# forest = RandomForestClassifier( n_estimators = 100 )

# print ("Fitting a random forest to labeled training data...")
# forest = forest.fit( trainDataVecs, train["sentiment"] )

# # Test & extract results 
# result = forest.predict( testDataVecs )

# # Write the test results 
# output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
# output.to_csv( "result_03.csv", index=False, quoting=3 )

In [79]:
from sklearn.cluster import KMeans
import time


In [67]:
start = time.time()
word_vertors = model.wv.syn0
num_clusters =int( word_features.shape[0] / 5)
kmeans_clustering = KMeans(n_clusters=int(num_clusters))
idx = kmeans_clustering.fit_predict(word_features)
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  0.00021886825561523438 seconds.


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [62]:
# For the first 10 clusters
word_centroid_map = dict(zip( model.wv.index2word, idx ))

for cluster in range(0,10):
#     #
#     # Print the cluster number  
    print("Cluster %d" % cluster)
#     #
#     # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)

Cluster 0
['lamarr', 'hedy', 'olga']
Cluster 1
['service', 'staff', 'safety', 'demand', 'firm', 'insurance', 'services', 'protection', 'reward', 'request', 'employment', 'ransom', 'institute', 'escort', 'volunteer', 'debts', 'payment', 'arrange', 'taxes']
Cluster 2
['jealous', 'scheming', 'bitchy', 'conniving', 'distraught', 'timid', 'virginal', 'headstrong', 'tomboy', 'plucky']
Cluster 3
['rear', 'statue', 'closeup', 'silhouette', 'parting', 'nary']
Cluster 4
['fabric']
Cluster 5
['landmark', 'quintessential', 'monumental', 'eklavya', 'triumphant', 'nadir', 'marking', 'crowning', 'purest']
Cluster 6
['alex', 'mills', 'hector', 'edison', 'wolverine', 'cabot', 'hatton', 'duryea', 'sasha', 'merrill', 'sloan', 'bannister', 'mantee', 'mendoza']
Cluster 7
['tearing', 'stepping', 'pouring', 'poured', 'tripping', 'rubble', 'melted', 'crawls']
Cluster 8
['calling', 'helping', 'teaches', 'insults', 'encourages', 'inviting', 'attracts', 'manipulating', 'interviewing', 'enables', 'prompts', 'info

In [68]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    num_centroids = max(list(word_centroid_map.values())) + 1
    bag_of_centroids = np.zeros(num_centroids, dtype='float32')
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [70]:
train_cetroids = np.zeros((train["review"].size, num_clusters), dtype="float32")

In [74]:
counter = 0
for review in clean_train_reviews:
    train_cetroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [75]:
test_cetroids = np.zeros((test["review"].size, num_clusters), dtype="float32")
counter = 0
for review in clean_test_reviews:
    test_cetroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [82]:
# Fit a random forest and extract predictions 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_cetroids,train["sentiment"])
result = forest.predict(test_cetroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "result_03_2.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
